In [24]:
from scipy.special import lpmv
from scipy.special import factorial as fact
from numpy import sqrt
from numpy import cos
from numpy import sin 
import numpy as np 
from pandas import read_csv

In [25]:
coeff_file_name = './coeff_rate/mit_coeff_rates_ALRT.txt'
lat = float(82.4942939716) 
lon = float(-62.3404664131) 
max_deg = 80
colat = ( 90 - lat )*np.pi/180.0 
lon = ( lon - 360.0*float(lon > 180.0) )*np.pi/180.0 

In [26]:
load_love_file_name = 'Load_Love2_CM.dat'
load_love_pd_dataframe =  read_csv(load_love_file_name, header = None, skiprows = 13, sep = '\s+',
        dtype = str)
load_love_pd_dataframe[1] = load_love_pd_dataframe[1].str.replace('D','E')
load_love_pd_dataframe[2] = load_love_pd_dataframe[2].str.replace('D','E')
load_love_pd_dataframe[3] = load_love_pd_dataframe[3].str.replace('D','E')
load_love_data = np.asarray(load_love_pd_dataframe,
        dtype = float)
h = load_love_data[:,1]
k = load_love_data[:,3]

In [27]:
radius_of_earth = 6378100.0     # in meters m
Fv = radius_of_earth*(1.1677 * load_love_data[:,0]  - 0.5233)     # Fv values depends on the degree, first column of load_love_data is degree
Fe =  radius_of_earth * h/(1+k) # This has inf and nan
Fe[0] = 0   # in CM frame degree 0 and 1 cannot be calculated  
Fe[1] = 0

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
def nalf(n,m,x):
    # order degree input
    return lpmv(m,n,x) * (-1)**m * sqrt( (2-float(m==0)) * (2.0*n + 1) * fact(n-m)/fact(n+m) )

In [29]:
data = np.asarray( read_csv( coeff_file_name , header = None , skiprows = 0 , sep = '\s+' ) ) 
n = data[:,0].astype(int)
m = data[:,1].astype(int)
C = data[:,2].astype(float)
S = data[:,3].astype(float)

In [30]:
elastic = 0.0
visco_elastic = 0.0
geoid = 0.0
for i in range( np.size(n)): 
    elastic         += nalf( n[i] , m[i] , cos(colat) ) * Fe[n[i]] * ( C[i]*cos(m[i]*lon) + S[i]*sin(m[i]*lon) ) 
    #visco_elastic   += nalf( n[i] , m[i] , cos(colat) ) * Fv[n[i]] * ( C[i]*cos(m[i]*lon) + S[i]*sin(m[i]*lon) ) 
    visco_elastic   += nalf( n[i] , m[i] , cos(colat) ) * radius_of_earth*(1.1677*n[i] - 0.5233) * ( C[i]*cos(m[i]*lon) + S[i]*sin(m[i]*lon) ) 
    #visco_elastic   += nalf( n[i] , m[i] , cos(colat) ) * radius_of_earth*(2.0*n[i] + 1.0) * ( C[i]*cos(m[i]*lon) + S[i]*sin(m[i]*lon) ) 
    geoid           += nalf( n[i] , m[i] , cos(colat) ) * radius_of_earth * ( C[i]*cos(m[i]*lon) + S[i]*sin(m[i]*lon) ) 

In [55]:
nalf(5,5,cos(colat)) * (Fv[5] - Fe[5]) * sin(5*lon)#*1000* 80*10**-12 

2751.0308540230753

In [47]:
A = np.asmatrix(np.array([[4,7],[2,6]]))

In [49]:
b = np.asmatrix(np.array([[1],[2]]))

In [52]:
print(A,b,A*b)

[[4 7]
 [2 6]] [[1]
 [2]] [[18]
 [14]]


In [54]:
A*(A.T * A).I * A.T * b

matrix([[1.],
        [2.]])